In [142]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
import traceback

In [550]:
pd.set_option('display.max_columns',30)

pd.set_option('display.max_rows',500)

In [116]:
# Input to the function should be (DB server, new DB name, user, pwd)
# This function will create new DB using existing DB and setup the connection with returning cur

def create_new_db(default_db, new_db, user, pwd):
    try: 
        with connection(default_db) as (conn, cur):
            cur.execute(f"DROP DATABASE IF EXISTS {Name}")
            cur.execute(f"CREATE DATABASE {Name}")
    except psycopg2.Error as e:
        print(e)

In [115]:
# This function will take cursor and List of attributes as a input.
# List should be in format ["ID INT", "Name VARCHAR(20)"]

def createTable(List, db_name, tableName):
    try:
        with connection(db_name) as (conn, cur):
            string = ""
            for i in List:
                string+= f"{i},"
            string = string[:-1]
            cur.execute(f"CREATE TABLE IF NOT EXISTS {tableName} ({string})")
            
    except psycopg2.Error as e:
        print(e)

In [175]:
from contextlib import contextmanager

@contextmanager
def connection(db_name):
    try: 
        conn_string = f"host=localhost dbname={db_name} user=postgres password=root"
        conn = psycopg2.connect(conn_string)
        cur = conn.cursor()
        yield (conn, cur)
        
    except Exception as e:
        traceback.print_tb(e.__traceback__)
        traceback.print_exc()
        print(e)
    
    finally :
        conn.commit()
        cur.close()
        conn.close()

In [322]:
def sql(db_name,query):
    with connection(db_name) as (conn, cur):
        cur.execute(query)
        if 'select' in query.lower():
            ret = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            DF = pd.DataFrame(ret, columns=column_names)
            return DF
    return "Done"

# Inserting bulk data in database


Basic Syntax:
https://www.postgresqltutorial.com/postgresql-tutorial/postgresql-insert-multiple-rows/

Comparision of different methods:

https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/

Future purpose for comparision of different methods:
https://github.com/NaysanSaran/pandas2postgresql/blob/master/notebooks/Psycopg2_Bulk_Insert_Speed_Benchmark.ipynb


Motivation to use cur.mogrify: 
Basically, executeMany will run SQL query for each record and cur.mogrify will return string as SQL query.

https://stackoverflow.com/questions/8134602/psycopg2-insert-multiple-rows-with-one-query
https://www.geeksforgeeks.org/python-psycopg2-insert-multiple-rows-with-one-query/


What is cursor:

The Cursor class of the psycopg library provide methods to execute the PostgreSQL commands in the database using python code. Using the methods of it you can execute SQL statements, fetch data from the result sets, call procedures. You can create Cursor object using the cursor() method of the Connection object/class

Exception tracback:

https://www.w3docs.com/snippets/python/how-do-i-print-an-exception-in-python.html#:~:text=You%20can%20print%20an%20exception,variable%20name%20for%20an%20exception.&text=You%20can%20also%20use%20the,detailed%20traceback%20of%20the%20exception.

numpy datatype solution:
https://stackoverflow.com/questions/50626058/psycopg2-cant-adapt-type-numpy-int64
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_records.html


In [176]:
def inserting_value(DF,db_name,table_name):
    try:
        # Basically we need list of tuples of records. We can not use DF.to_record function as it returned numpy objects
        # Which are not suitable here
        list_of_tuple = [tuple(row) for row in DF.itertuples(index=False)]
        
        # to get lenght of row
        length = len(list_of_tuple[0])

        # Just to get no. %s 
        string_format = (",").join(["%s"] * length)
        
        with connection(db_name) as (conn, cur):

            # get tuples in string format
            args = ','.join(cur.mogrify(f"({string_format})", i).decode('utf-8')
                for i in list_of_tuple)

            # real query
            query = f"INSERT INTO {table_name} VALUES " + (args)
            cur.execute(query)
        
    except Exception as e:
        traceback.print_tb(e.__traceback__)

In [6]:
import unittest
from mod_amr import inserting_value
from unittest.mock import patch, MagicMock
import pandas as pd

class test_mod(unittest.TestCase):
    
    @patch(mod_amr.connection)
    def test_insert(self, mock_connection):
        
        mock_conn = MagicMock()
        mock_cur = MagicMock()
        mock_cur.mogrify.return_value = [(),()]
        mock_cur.execute.return_value = None
        
        A = {'col1': [1,2],
            'col2':['A','B']}
        B = pd.DataFrame(A)
        
        mock_connection.return_value = (mock_conn, mock_cur)
        
        self.assertEqual(inserting_value(B,'random','table'),"INSERT INTO table VALUES (1,'A'), (2,'B')")

In [496]:
from collections import defaultdict

def Len_prod(x):
    try:
        if type(x) == float:
            return 0
        else:
            return len(x)
    except:
        return 0


def suggested_schema(DF):
    Map = defaultdict(lambda : "Define")
    Map['int64'] = 'INT' 
    Map['int32'] = 'INT' 
    Map['object'] ='VARCHAR'
    Map['float64'] ='double precision'
    Map['datetime64[ns]'] = 'Date'
    col = list(DF.columns)
    schema = ""
    schema_list = []
    for i in range(len(col)):
        try:
            if Map[str(list(DF.dtypes)[i])] == 'VARCHAR':
                #print(Map[str(list(DF.dtypes)[i])], col[i])
                schema = col[i] + ' ' + Map[str(list(DF.dtypes)[i])] + '(' + str(DF[col[i]].apply(Len_prod).max()) + ')' 
            else:
                #print(Map[str(list(DF.dtypes)[i])], col[i])
                schema = col[i] + ' ' + Map[str(list(DF.dtypes)[i])]
            schema_list.append(schema)
        except:
            schema_list.append(col[i] + ' ' + 'Excepted col')
            continue
    return schema_list

# suggested_schema(result)

# str(DF[col[i]].apply(len).max()) --> Apply len function on every element of series object and then we find max of that list.
# Map[str(list(DF.dtypes)[i])] --> DF.dtypes numpy object which we need to convert into string.
# Len_prod was made for handling null values

In [518]:
def date_modification(x):
    Y = x[:6]
    if int(x[6:]) >= 50 and int(x[6:]) <= 99:
        Y = Y + '19' + x[6:]
    else:
        Y = Y + '20' + x[6:]
    return Y

In [118]:
## Creating circuit DF in pandas

circuits = pd.read_csv("C:\\Users\\91942\\Desktop\\Big Data\\DataSets\\archive\\circuits.csv")

In [207]:
pd.DataFrame(circuits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   79 non-null     int64  
 1   circuitRef  79 non-null     object 
 2   name        79 non-null     object 
 3   location    79 non-null     object 
 4   country     79 non-null     object 
 5   lat         79 non-null     float64
 6   lng         79 non-null     float64
 7   alt         79 non-null     int32  
 8   url         79 non-null     object 
dtypes: float64(2), int32(1), int64(1), object(5)
memory usage: 5.4+ KB


""


In [98]:
circuitSchema = ["circuitId INT PRIMARY KEY", "circuitRef VARCHAR(25)", "name VARCHAR(50)", "location VARCHAR(50)", "country VARCHAR(20)", "lat double precision", "lng double precision", "alt INT", "url VARCHAR(100)"]

In [117]:
createTable(circuitSchema,"formulaone","circuits")

In [121]:
circuits['alt'].unique()

array(['10', '18', '7', '109', '130', '13', '228', '153', '103', '264',
       '4', '401', '162', '583', '5', '785', '223', '578', '37', '45',
       '678', '3', '8', '266', '58', '1460', '88', '2227', '345', '432',
       '0', '1126', '177', '145', '6', '36', '484', '139', '12', '639',
       '609', '485', '332', '79', '790', '214', '81', '67', '15', '676',
       '20', '28', '470', '53', '158', '19', '129', '551', '85', '194',
       '161', '2', '-7', '9', '108', '255', '\\N'], dtype=object)

In [123]:
circuits.loc[circuits['alt'] == '\\N']

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
77,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,\N,http://en.wikipedia.org/wiki/Losail_Internatio...
78,79,miami,Miami International Autodrome,Miami,USA,25.9581,-80.2389,\N,http://en.wikipedia.org/wiki/Miami_Internation...


In [127]:
circuits['alt'] = circuits['alt'].replace({'\\N':2}).astype('int')

In [128]:
circuits['alt'].unique()

array([  10,   18,    7,  109,  130,   13,  228,  153,  103,  264,    4,
        401,  162,  583,    5,  785,  223,  578,   37,   45,  678,    3,
          8,  266,   58, 1460,   88, 2227,  345,  432,    0, 1126,  177,
        145,    6,   36,  484,  139,   12,  639,  609,  485,  332,   79,
        790,  214,   81,   67,   15,  676,   20,   28,  470,   53,  158,
         19,  129,  551,   85,  194,  161,    2,   -7,    9,  108,  255])

In [129]:
circuits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   79 non-null     int64  
 1   circuitRef  79 non-null     object 
 2   name        79 non-null     object 
 3   location    79 non-null     object 
 4   country     79 non-null     object 
 5   lat         79 non-null     float64
 6   lng         79 non-null     float64
 7   alt         79 non-null     int32  
 8   url         79 non-null     object 
dtypes: float64(2), int32(1), int64(1), object(5)
memory usage: 5.4+ KB


In [131]:
# To record is used to convert DataFrame to list of tuple..

circuits_tuple = circuits.to_records()

In [177]:
# inserting_value(circuits,'formulaone','circuits')

In [190]:
sql('formulaone',"SELECT circuitid FROM circuits")

,circuitid
0,1
1,2
2,3
3,4
4,5
...,...
74,75
75,76
76,77
77,78


In [520]:
races = pd.read_csv("C:\\Users\\91942\\Desktop\\Big Data\\DataSets\\archive\\races.csv")

In [521]:
races

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,29/03/09,6:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,05/04/09,9:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,19/04/09,7:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,26/04/09,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,10/05/09,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...,...
1074,1092,2022,18,22,Japanese Grand Prix,09/10/22,5:00:00,http://en.wikipedia.org/wiki/2022_Japanese_Gra...
1075,1093,2022,19,69,United States Grand Prix,23/10/22,19:00:00,http://en.wikipedia.org/wiki/2022_United_State...
1076,1094,2022,20,32,Mexico City Grand Prix,30/10/22,20:00:00,http://en.wikipedia.org/wiki/2022_Mexican_Gran...
1077,1095,2022,21,18,Brazilian Grand Prix,13/11/22,18:00:00,http://en.wikipedia.org/wiki/2022_Brazilian_Gr...


In [522]:
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   raceId     1079 non-null   int64 
 1   year       1079 non-null   int64 
 2   round      1079 non-null   int64 
 3   circuitId  1079 non-null   int64 
 4   name       1079 non-null   object
 5   date       1079 non-null   object
 6   time       1079 non-null   object
 7   url        1079 non-null   object
dtypes: int64(4), object(4)
memory usage: 67.6+ KB


In [523]:
## Strictly execute this once after importing Dataframe

races['date'] = races['date'].apply(date_modification)

In [525]:
# list(races['date'].unique())

In [526]:
races['date'] = pd.to_datetime(races['date'], format="%d/%m/%Y")
races['date']

0      2009-03-29
1      2009-04-05
2      2009-04-19
3      2009-04-26
4      2009-05-10
          ...    
1074   2022-10-09
1075   2022-10-23
1076   2022-10-30
1077   2022-11-13
1078   2022-11-20
Name: date, Length: 1079, dtype: datetime64[ns]

In [528]:
races.loc[races['time'] == '\\N', 'time']

89     \N
90     \N
91     \N
92     \N
93     \N
       ..
834    \N
835    \N
836    \N
837    \N
838    \N
Name: time, Length: 731, dtype: object

In [529]:
races['time'] = races['time'].replace("\\N","15:00:00")

#### Problem with time

https://stackoverflow.com/questions/56816833/pandas-pd-to-datetime-only-keep-time-do-not-date

In [530]:
races['time'] = pd.to_datetime(races['time'],format="%H:%M:%S").dt.time
races['time'] = races['time'].astype(str)

In [531]:
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   raceId     1079 non-null   int64         
 1   year       1079 non-null   int64         
 2   round      1079 non-null   int64         
 3   circuitId  1079 non-null   int64         
 4   name       1079 non-null   object        
 5   date       1079 non-null   datetime64[ns]
 6   time       1079 non-null   object        
 7   url        1079 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 67.6+ KB


In [532]:
print(suggested_schema(races))

['raceId INT', 'year INT', 'round INT', 'circuitId INT', 'name VARCHAR(29)', 'date Date', 'time VARCHAR(8)', 'url VARCHAR(63)']


In [534]:
race_schema = ['raceId INT PRIMARY KEY', 'year INT', 'round INT', 'circuitId INT REFERENCES circuits(circuitId)', 'name VARCHAR(35)', 'date Date', 'time VARCHAR(8)', 'url VARCHAR(70)']

In [311]:
createTable(race_schema,"formulaone","races")

In [319]:
sql("formulaone", """SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_name = 'races'""")

,table_name,column_name,data_type
0,races,date,date
1,races,round,integer
2,races,raceid,integer
3,races,circuitid,integer
4,races,year,integer
5,races,url,character varying
6,races,name,character varying
7,races,time,character varying


In [324]:
# sql("formulaone", """ALTER TABLE races ALTER COLUMN name TYPE VARCHAR(35)""")

'Done'

In [535]:
inserting_value(races, 'formulaone', 'races')

In [327]:
sql("formulaone", """SELECT * FROM races""")

,raceid,year,round,circuitid,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-05-04,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-10-05,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...,...
1074,1092,2022,18,22,Japanese Grand Prix,2022-09-10,05:00:00,http://en.wikipedia.org/wiki/2022_Japanese_Gra...
1075,1093,2022,19,69,United States Grand Prix,2022-10-23,19:00:00,http://en.wikipedia.org/wiki/2022_United_State...
1076,1094,2022,20,32,Mexico City Grand Prix,2022-10-30,20:00:00,http://en.wikipedia.org/wiki/2022_Mexican_Gran...
1077,1095,2022,21,18,Brazilian Grand Prix,2022-11-13,18:00:00,http://en.wikipedia.org/wiki/2022_Brazilian_Gr...


In [328]:
# sql("formulaone", """DROP TABLE circuits1""")

'Done'

In [ ]:
## Retired calculations


# races['name'].apply(len).max()

In [392]:
# Transformations Done are as follows:
# 1. Changed \\N to 72 and finally data type of Number to Int
# 2. Code col modified with surname 3 initials


drivers = pd.read_csv("C:\\Users\\91942\\Desktop\\Big Data\\DataSets\\archive\\drivers.csv", parse_dates=['dob'])

In [393]:
drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   driverId     854 non-null    int64         
 1   driverRef    854 non-null    object        
 2   number       854 non-null    object        
 3   code         854 non-null    object        
 4   forename     854 non-null    object        
 5   surname      854 non-null    object        
 6   dob          854 non-null    datetime64[ns]
 7   nationality  854 non-null    object        
 8   url          854 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 60.2+ KB


In [394]:
drivers.head(1)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


In [395]:
drivers['number'].unique()

array(['44', '\\N', '6', '14', '7', '88', '19', '99', '22', '5', '8',
       '10', '27', '13', '11', '3', '25', '4', '21', '77', '17', '20',
       '26', '45', '9', '28', '33', '12', '55', '98', '53', '30', '94',
       '2', '31', '18', '16', '35', '63', '23', '51', '89', '47', '24'],
      dtype=object)

In [396]:
drivers['number'].replace('\\N', 72, inplace=True)
drivers['number'] = drivers['number'].astype(int)

In [397]:
drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   driverId     854 non-null    int64         
 1   driverRef    854 non-null    object        
 2   number       854 non-null    int32         
 3   code         854 non-null    object        
 4   forename     854 non-null    object        
 5   surname      854 non-null    object        
 6   dob          854 non-null    datetime64[ns]
 7   nationality  854 non-null    object        
 8   url          854 non-null    object        
dtypes: datetime64[ns](1), int32(1), int64(1), object(6)
memory usage: 56.8+ KB


In [398]:
drivers.loc[drivers['code'] == '\\N','code'] = drivers.loc[drivers['code'] == '\\N']['surname'].apply(lambda x : x[:3].upper() if len(x[:3].upper().strip()) == 3 else x[:4].upper().replace(" ", ""))

In [399]:
drivers.loc[drivers['code'] == '\\N','code']

Series([], Name: code, dtype: object)

In [407]:
suggested_schema(drivers)

['driverId INT',
 'driverRef VARCHAR(18)',
 'number INT',
 'code VARCHAR(3)',
 'forename VARCHAR(17)',
 'surname VARCHAR(23)',
 'dob Date',
 'nationality VARCHAR(17)',
 'url VARCHAR(83)']

In [408]:
driver_schema = ['driverId INT PRIMARY KEY','driverRef VARCHAR(18)', 'number INT','code VARCHAR(3)', 'forename VARCHAR(17)','surname VARCHAR(23)','dob Date','nationality VARCHAR(17)',
 'url VARCHAR(83)']

In [409]:
createTable(driver_schema,"formulaone","drivers")

In [410]:
inserting_value(drivers,"formulaone", "drivers")

In [411]:
sql("formulaone", "SELECT * FROM drivers")

,driverid,driverref,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,72,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,72,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
849,851,aitken,89,AIT,Jack,Aitken,1995-09-23,British,http://en.wikipedia.org/wiki/Jack_Aitken
850,852,tsunoda,22,TSU,Yuki,Tsunoda,2000-05-11,Japanese,http://en.wikipedia.org/wiki/Yuki_Tsunoda
851,853,mazepin,9,MAZ,Nikita,Mazepin,1999-03-02,Russian,http://en.wikipedia.org/wiki/Nikita_Mazepin
852,854,mick_schumacher,47,MSC,Mick,Schumacher,1999-03-22,German,http://en.wikipedia.org/wiki/Mick_Schumacher


In [406]:
# Retired Functions

# Used
# drivers.loc[drivers['code'] == '\\N']['surname'].apply(lambda x : x[:3].upper() if len(x[:3].upper(). strip()) == 3 else x[:4].upper().replace(" ", ""))


# Helped to see where error happend
# def Len(x):
#     print(x)
#     return len(x)


# snippet from suggested schema
# str(drivers['code'].apply(Len).max())


# Just checked and everything is of typr object... Not actually necessary
# filt = drivers['code'].apply(lambda x: isinstance(x,object))
# drivers.loc[~filt]


# This is very Helpful for checking where is the error happend
# list(drivers['code'].unique())

In [444]:
na_value_cust = ['\\N']
result = pd.read_csv("C:\\Users\\91942\\Desktop\\Big Data\\DataSets\\archive\\results.csv", na_values=na_value_cust)

In [445]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25460 entries, 0 to 25459
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         25460 non-null  int64  
 1   raceId           25460 non-null  int64  
 2   driverId         25460 non-null  int64  
 3   constructorId    25460 non-null  int64  
 4   number           25454 non-null  float64
 5   grid             25460 non-null  int64  
 6   position         14663 non-null  float64
 7   positionText     25460 non-null  object 
 8   positionOrder    25460 non-null  int64  
 9   points           25460 non-null  float64
 10  laps             25460 non-null  int64  
 11  time             6850 non-null   object 
 12  milliseconds     6849 non-null   float64
 13  fastestLap       7010 non-null   float64
 14  rank             7211 non-null   float64
 15  fastestLapTime   7010 non-null   object 
 16  fastestLapSpeed  7010 non-null   float64
 17  statusId    

In [505]:
result.loc[result['points'] == 0,'position']

8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
         ... 
25455    16.0
25456    17.0
25457     NaN
25458     NaN
25459     NaN
Name: position, Length: 17940, dtype: float64

In [536]:
result_races = result.merge(right=races ,right_on='raceId', left_on='raceId', how='inner')

In [516]:
list(races['date'].unique())

[numpy.datetime64('2009-03-29T00:00:00.000000000'),
 numpy.datetime64('2009-05-04T00:00:00.000000000'),
 numpy.datetime64('2009-04-19T00:00:00.000000000'),
 numpy.datetime64('2009-04-26T00:00:00.000000000'),
 numpy.datetime64('2009-10-05T00:00:00.000000000'),
 numpy.datetime64('2009-05-24T00:00:00.000000000'),
 numpy.datetime64('2009-07-06T00:00:00.000000000'),
 numpy.datetime64('2009-06-21T00:00:00.000000000'),
 numpy.datetime64('2009-12-07T00:00:00.000000000'),
 numpy.datetime64('2009-07-26T00:00:00.000000000'),
 numpy.datetime64('2009-08-23T00:00:00.000000000'),
 numpy.datetime64('2009-08-30T00:00:00.000000000'),
 numpy.datetime64('2009-09-13T00:00:00.000000000'),
 numpy.datetime64('2009-09-27T00:00:00.000000000'),
 numpy.datetime64('2009-04-10T00:00:00.000000000'),
 numpy.datetime64('2009-10-18T00:00:00.000000000'),
 numpy.datetime64('2009-01-11T00:00:00.000000000'),
 numpy.datetime64('2008-03-16T00:00:00.000000000'),
 numpy.datetime64('2008-03-23T00:00:00.000000000'),
 numpy.datet

In [510]:
result_races.head(1)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name,date,time_y,url
0,1,18,1,1,22.0,1,1.0,1,1,10.0,...,1:27.452,218.3,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...


In [538]:
result_races['date'].max()

Timestamp('2022-04-10 00:00:00')

In [549]:
pd.set_option('display.max_columns',30)

pd.set_option('display.max_rows',500)

result_races[(result_races['date'].dt.year >= 1950) & (result_races['date'].dt.year < 1951)]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time_x,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name,date,time_y,url
20140,20025,833,642,51,2.0,1,1.0,1,1,9.0,70,2:13:23.6,8003600.0,NaN,NaN,NaN,NaN,1,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20141,20026,833,786,51,3.0,2,2.0,2,2,6.0,70,+2.6,8006200.0,NaN,NaN,NaN,NaN,1,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20142,20027,833,686,51,4.0,4,3.0,3,3,4.0,70,+52.0,8055600.0,NaN,NaN,NaN,NaN,1,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20143,20028,833,704,154,14.0,6,4.0,4,4,3.0,68,NaN,NaN,NaN,NaN,NaN,NaN,12,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20144,20029,833,627,154,15.0,9,5.0,5,5,2.0,68,NaN,NaN,NaN,NaN,NaN,NaN,12,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20145,20030,833,619,151,12.0,13,6.0,6,6,0.0,67,NaN,NaN,NaN,NaN,NaN,NaN,13,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20146,20031,833,787,151,11.0,15,7.0,7,7,0.0,67,NaN,NaN,NaN,NaN,NaN,NaN,13,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20147,20032,833,741,154,16.0,14,8.0,8,8,0.0,65,NaN,NaN,NaN,NaN,NaN,NaN,15,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20148,20033,833,784,105,6.0,16,9.0,9,9,0.0,64,NaN,NaN,NaN,NaN,NaN,NaN,16,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...
20149,20034,833,778,105,10.0,20,10.0,10,10,0.0,64,NaN,NaN,NaN,NaN,NaN,NaN,16,1950,1,9,British Grand Prix,1950-05-13,15:00:00,http://en.wikipedia.org/wiki/1950_British_Gran...


### Helped resources

Function which can be applied on 2 DFs

https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

https://stackoverflow.com/questions/67805424/problem-with-if-conditional-with-none-variable-in-dataframe-python

In [551]:
import numpy as np
List = [25,18,15,12,10,8,6,4,2,1] + [0 for i in range(25)]
Map = {float(i):List[i-1] for i in range(1,35)}
Map[0.0] = 0


def unified_system(x):
    final_point = 0
    
    # Actually there is no need of this since we fill nan values with 0
    if x[6] == np.nan or x[6] == None or x[6] is np.nan or x[6] == "NaN" or x[6] == "nan":
        return final_point
    final_point = Map[x[6]]
    if x[14] == 1 and x[6] <= 10.0:
        return final_point + 1
    return final_point

In [552]:
values = {'position':0.0}
result.fillna(value=values,inplace=True)

In [553]:
result['unified_points'] = result.apply(lambda x: unified_system(x) ,axis=1)

In [555]:
# Tool to check point for each race

result.loc[result['raceId'] == 834][['position','rank','unified_points']]

,position,rank,unified_points
20045,1.0,NaN,25
20046,2.0,NaN,18
20047,3.0,NaN,15
20048,4.0,NaN,12
20049,5.0,NaN,10
20050,6.0,NaN,8
20051,7.0,NaN,6
20052,0.0,NaN,0
20053,0.0,NaN,0
20054,0.0,NaN,0


In [556]:
suggested_schema(result)

['resultId INT',
 'raceId INT',
 'driverId INT',
 'constructorId INT',
 'number double precision',
 'grid INT',
 'position double precision',
 'positionText VARCHAR(2)',
 'positionOrder INT',
 'points double precision',
 'laps INT',
 'time VARCHAR(11)',
 'milliseconds double precision',
 'fastestLap double precision',
 'rank double precision',
 'fastestLapTime VARCHAR(8)',
 'fastestLapSpeed double precision',
 'statusId INT',
 'unified_points INT']

In [558]:
result_scheama = ['resultId INT PRIMARY KEY',
 'raceId INT REFERENCES races(raceId) ',
 'driverId INT REFERENCES drivers(driverId)',
 'constructorId INT',
 'number double precision',
 'grid INT',
 'position double precision',
 'positionText VARCHAR(5)',
 'positionOrder INT',
 'points double precision',
 'laps INT',
 'time VARCHAR(20)',
 'milliseconds double precision',
 'fastestLap double precision',
 'rank double precision',
 'fastestLapTime VARCHAR(25)',
 'fastestLapSpeed double precision',
 'statusId INT',
 'unified_points INT']

In [559]:
createTable(result_scheama,"formulaone","result")

In [560]:
inserting_value(result,"formulaone", "result")

In [575]:
sql("formulaone", """SELECT  r.points as everPoints, CONCAT(d.forename,' ', d.surname) as driverName FROM (SELECT driverId, SUM(unified_points) as points FROM result
    GROUP BY driverId) r JOIN drivers d ON r.driverId = d.driverId ORDER BY points DESC""")

,everpoints,drivername
0,4592,Lewis Hamilton
1,3910,Michael Schumacher
2,3288,Sebastian Vettel
3,2843,Fernando Alonso
4,2830,Kimi Räikkönen
...,...,...
849,0,Antonio Creus
850,0,Sergey Sirotkin
851,0,Geoff Duke
852,0,Robert Doornbos


In [577]:
sql("formulaone", """SELECT driverId, count(*) as wins FROM result WHERE position = 1.0 GROUP BY driverId ORDER BY wins DESC""")

,driverid,wins
0,1,103
1,30,91
2,20,53
3,117,51
4,102,41
5,4,32
6,95,31
7,328,27
8,373,25
9,182,25


In [581]:
# Q. We need to find no. of unique drivers who won the races
# -> 


sql("formulaone", """SELECT count(*) as drivers FROM (SELECT driverId, count(*) as wins FROM result WHERE position = 1.0 GROUP BY driverId ORDER BY wins DESC) v""")

,drivers
0,111


In [587]:
sql("formulaone", """SELECT *
                    FROM result r JOIN races ra 
                    ON r.raceId = ra.raceId
                    WHERE ra.date = (SELECT MAX(date) FROM races)""")

,resultid,raceid,driverid,constructorid,number,grid,position,positiontext,positionorder,points,laps,time,milliseconds,fastestlap,rank,fastestlaptime,fastestlapspeed,statusid,unified_points,raceid,year,round,circuitid,name,date,time,url
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1,25,18,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,+5.478,5696094.0,41.0,3.0,1:27.739,217.586,1,18,18,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,+8.163,5698779.0,41.0,5.0,1:28.090,216.719,1,15,18,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,+17.181,5707797.0,58.0,7.0,1:28.603,215.464,1,12,18,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,+18.014,5708630.0,43.0,1.0,1:27.418,218.385,1,11,18,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25455,25461,1076,849,3,6.0,18,16.0,16,16,0.0,57,NaN,NaN,49.0,18.0,1:23.882,226.518,11,0,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...
25456,25462,1076,4,214,14.0,10,17.0,17,17,0.0,57,NaN,NaN,57.0,2.0,1:20.846,235.024,11,0,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...
25457,25463,1076,830,9,1.0,2,0.0,R,18,0.0,38,NaN,NaN,37.0,6.0,1:21.677,232.633,9,0,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...
25458,25464,1076,20,117,5.0,17,0.0,R,19,0.0,22,NaN,NaN,17.0,19.0,1:25.189,223.042,3,0,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...


In [ ]:
# Retired calculations:



# Handling null values
# import numpy as np

# def Len_test(x):
#     try:
#         if type(x) == float:
#             return 0
#         else:
#             return len(x)
#     except:
#         return 0

# str(result['time'].apply(Len_test).max())

In [11]:
string_format = "%s,%s"
list_of_tuple = [(1,'A'),(2,'B')]
args__ = ','.join([f"({string_format})" % i for i in list_of_tuple])
args__

'(1,A),(2,B)'